<a href="https://colab.research.google.com/github/konductor000/Style-transfer-BERT/blob/main/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install datasets
!pip install evaluate
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.0 -O train_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.1 -O train_positive
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.0 -O dev_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.1 -O dev_positive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 2.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 6.6 MB/s eta 0:00:00


In [ ]:
!head -n 5 ./dev_positive
!echo
!head -n 5 ./dev_negative

staff behind the deli counter were super nice and efficient !
love this place !
the staff are always very nice and helpful .
the new yorker was amazing .
very ny style italian deli .

ok never going back to this place again .
easter day nothing open , heard about this place figured it would ok .
the host that walked us to the table and left without a word .
it just gets worse .
the food tasted awful .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = 'drive/MyDrive/saves/style_transfer/'

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cpu':
    print("Fine-tuning BERT without an accelerator is not party-approved.")

In [ ]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

def train_model(model, tokenizer, file_path, output_dir, epochs=1):
    print("Preparing the training data...")
    dataset = LineByLineTextDataset(
        file_path=file_path, tokenizer=tokenizer, block_size=128)

    print("Dataset ready!")

    trainer = Trainer(
        model=model, train_dataset=dataset, 
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=True, mlm_probability=0.15),
        args=TrainingArguments(
            output_dir=output_dir, overwrite_output_dir=True,
            num_train_epochs=epochs, per_device_train_batch_size=32,
            save_steps=10_000, save_total_limit=2),
    )

    trainer.train()

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

use_funetuned = True

if use_funetuned:
    bert_mlm_positive = BertForMaskedLM.from_pretrained(dir + 'positive')
    bert_mlm_negative = BertForMaskedLM.from_pretrained(dir + 'negative')
else:
    bert_mlm_positive = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True).to(device).train(True)
    dataset_path_pos = "./train_positive"
    output_dir_pos = "./bert_mlm_positive"

    bert_mlm_negative = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True).to(device).train(True)
    dataset_path_neg = "./train_negative"
    output_dir_neg = "./bert_mlm_negative"

    train_model(bert_mlm_positive, tokenizer, dataset_path_pos, output_dir_pos, epochs=4)
    train_model(bert_mlm_negative, tokenizer, dataset_path_neg, output_dir_neg, epochs=5)

bert_mlm_positive.to(device)
bert_mlm_negative.to(device)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
if not use_funetuned:
    bert_mlm_positive.save_pretrained(dir + 'positive')
    bert_mlm_negative.save_pretrained(dir + 'negative')

In [ ]:
id_token = {value: key for key, value in tokenizer.vocab.items()}

mask = tokenizer.mask_token
tok_mask = 103
mask_id = 6
sequence = 'The sound in the cinema was good !'

sequence = sequence.split()
sequence[mask_id] = mask
sequence = ' '.join(sequence)
print(sequence)
batch = tokenizer([sequence], padding=True, truncation=True, return_tensors='pt')
batch = {key: value.to(device) for key, value in batch.items()}
mask_id = batch['input_ids'][0].tolist().index(tok_mask)

logits = bert_mlm_positive(**batch).logits.cpu().data.numpy()[0, mask_id]
logits = bert_mlm_negative(**batch).logits.cpu().data.numpy()[0, mask_id]

top_k = np.argsort(-logits)[:5]
for i in top_k:
    print(id_token[i])

The sound in the cinema was [MASK] !
horrible
terrible
awful
loud
ridiculous


In [ ]:
def get_replacements(sentence: str, num_tokens, k_best, epsilon=1e-3):
    """
    - split the sentence into tokens using the INGSOC-approved BERT tokenizer
    - find :num_tokens: tokens with the highest ratio (see above)
    - replace them with :k_best: words according to bert_mlm_positive
    :return: a list of all possible strings (up to k_best * num_tokens)
    """
    
    id_token = {value: key for key, value in tokenizer.vocab.items()}

    out_list = []
    sent_split = sentence.split()
    all_scores = []
    mask = tokenizer.mask_token
    tok_mask = 103

    for i in range(len(sent_split)):
        mask_id = i

        sequence = sent_split.copy()
        sequence[mask_id] = mask
        sequence = ' '.join(sequence)

        batch = tokenizer([sequence], padding=True, truncation=True, return_tensors='pt')
        batch = {key: value.to(device) for key, value in batch.items()}
        mask_id = batch['input_ids'][0].tolist().index(tok_mask)

        logits_pos = bert_mlm_positive(**batch).logits.cpu().data.numpy()[0, mask_id]
        logits_neg = bert_mlm_negative(**batch).logits.cpu().data.numpy()[0, mask_id]

        if sent_split[i] not in tokenizer.vocab:
            continue
        token = tokenizer.vocab[sent_split[i]]

        score = (logits_pos[token] + epsilon) / (logits_neg[token] + epsilon)

        all_scores.append(score)

    idx_list = np.argsort(all_scores)[:num_tokens]

    for i in idx_list:
        mask_id = i

        sequence = sent_split.copy()
        sequence[mask_id] = mask
        sequence = ' '.join(sequence)

        batch = tokenizer([sequence], padding=True, truncation=True, return_tensors='pt')
        batch = {key: value.to(device) for key, value in batch.items()}
        mask_id = batch['input_ids'][0].tolist().index(tok_mask)

        logits_pos = bert_mlm_positive(**batch).logits.cpu().data.numpy()[0, mask_id]

        top_k = np.argsort(-logits_pos)[:k_best]
        
        for token_id in top_k:
            out_seq = sent_split.copy()
            out_seq[i] = id_token[token_id]
            out_list.append(" ".join(out_seq))


    return out_list

In [ ]:
dev_data = list(open('./dev_negative'))

In [ ]:
dev_data[500:505]

['wrong !\n',
 "i 'm sure she wo n't issue a refund because - surprise !\n",
 "nothing special and the drinks are n't cheap .\n",
 'great wings and decent drinks but the wait staff is horrible !\n',
 'the tables are all saved by people crowded around the two tvs .\n']

In [ ]:
get_replacements("great wings and decent drinks but the wait staff is horrible !",
                 num_tokens=2, k_best=2)
# >>> ["great wings and decent drinks but the wait staff is great !", "great wings and decent drinks but the wait staff is awesome !"])

['great wings and decent drinks but the wait staff is awesome !',
 'great wings and decent drinks but the wait staff is great !',
 'great wings and decent drinks and the wait staff is horrible !',
 'great wings and decent drinks , the wait staff is horrible !']

In [ ]:
raw_dataset = []

for sentence in list(open('./dev_positive')):
    tokenized = tokenizer(sentence, padding=True, truncation=True)
    raw_dataset.append({**tokenized, 'labels': 1})

for sentence in list(open('./dev_negative')):
    tokenized = tokenizer(sentence, padding=True, truncation=True)
    raw_dataset.append({**tokenized, 'labels': 0})

In [ ]:
train_dataset = np.array(raw_dataset)

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, eval_dataset = train_test_split(train_dataset, test_size=0.15)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
use_finetuned = True

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

if not use_finetuned:
    trainer.train()
else:
    model = AutoModelForSequenceClassification.from_pretrained(
        dir + 'classicator', num_labels=2, id2label=id2label, label2id=label2id
    ) 

In [ ]:
if not use_finetuned:
    model.save_pretrained(dir + 'classicator')

In [ ]:
from transformers import pipeline

sentences = ['Very bad product!', 'Pretty good servings, and prices are high']

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(sentences)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'NEGATIVE', 'score': 0.9993836879730225},
 {'label': 'POSITIVE', 'score': 0.9998654127120972}]

In [ ]:
def take_res(result):
    output = []
    for i in range(len(result)):
        if result[i]['label'] == 'POSITIVE':
            output.append(result[i]['score'])
        else:
            output.append(1 - result[i]['score'])

    return output


def change_coment(text, num_tokens, k_best, num_iters = 5):
    
    beam_size = num_tokens * k_best

    beam_search = []
    beam_results = []

    beam_search += get_replacements(text, num_tokens, k_best)
    beam_results += take_res(classifier(beam_search))

    for i in range(num_iters):
        prob_text = []
        prob_score = []

        for j in range(beam_size):
            prob_text += get_replacements(beam_search[-j-1], num_tokens, k_best)
        
        prob_score += take_res(classifier(prob_text))
        new_branch = sorted([[prob_score[i], prob_text[i]] for i in range(len(prob_text))], reverse=True)

        for j in range(beam_size):
            beam_search.append(new_branch[j][1])
            beam_results.append(new_branch[j][0])

    beam = sorted(list(set([(beam_results[i], beam_search[i]) for i in range(len(beam_results))])) \
                  + [(take_res(classifier(text))[0], text)], reverse=True)

    return [pair[1] for pair in beam[:min(beam_size, len(beam))]]

In [ ]:
num_tokens = 2
k_best = 2
text = 'This place is weird and I do not like it at all !'.lower()

beam_search = change_coment(text, num_tokens, k_best)

In [ ]:
print(beam_search)

['this place is great and i do really like it at all !', 'this place is great and i do really like it after all !', 'this place is great and i do not like it after all !', 'this place is great and i do not like it at all !']
